# Prepare variables and S3 client

In [41]:
import os
import boto3
from botocore.client import Config

# helper variables
s3endpoint    = "http://minio-service:9000"
bucket_models = "production"
mount_path    = "./../models"

# Prepare S3 client
s3_client = boto3.client('s3',
        endpoint_url=s3endpoint,
        aws_access_key_id='minio',
        aws_secret_access_key='minio123',
        config=Config(signature_version='s3v4'),
        region_name='us-east-1')

# Get latest version available (locally)

In [42]:
path = mount_path+"/redbag"

latestversion = 0

for subdir, dirs, files in os.walk(path):
    dirs.sort()
    latestversion = dirs[-1]
    break
    
print(latestversion)

2


# Calculate version to push in S3 

In [43]:
objects = s3_client.list_objects_v2(Bucket=bucket_models)

version = 0

if "Contents" in objects:
    for obj in objects['Contents']:
        print(obj['Key'])
        s3version = int(obj['Key'].split('/')[2])
        if s3version > version:
            version = s3version
else:
    print("there are NO contents")
    
version += 1
version

models/redbag/1/fingerprint.pb
models/redbag/1/saved_model.pb
models/redbag/1/variables/variables.data-00000-of-00001
models/redbag/1/variables/variables.index


2

# Push latest version to S3

In [44]:
path = mount_path+"/redbag/"+latestversion

for subdir, dirs, files in os.walk(path):
    for file in files:
        full_path = os.path.join(subdir, file)
        s3_path = os.path.join(subdir, file)
        s3_path = s3_path.replace("/"+str(latestversion)+"/","/"+str(version)+"/")
        print(s3_path)
        s3_client.upload_file(full_path, bucket_models, s3_path)
        

models/redbag/2/saved_model.pb
models/redbag/2/fingerprint.pb
models/redbag/2/variables/variables.index
models/redbag/2/variables/variables.data-00000-of-00001
